# STE Binary Network

In [1]:
# import torch base code
import sys
from os import path
sys.path.append(path.join("..", "src"))
import base_torch

## Data

In [3]:
# imports
from base_torch.process_data import MfccDataset

train_dataset = MfccDataset(mfcc_dir='~/Music/test', transform=None)

AttributeError: 'list' object has no attribute 'read'